# Imports

In [40]:
# General

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Hyperparameter tuning
import optuna

# Models
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

# Data processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

# Data preparation

In [41]:
# Data input

try:
    data = pd.read_csv('../data/train.csv')
    data_to_predict = pd.read_csv('../data/test.csv')
    data_ccrisk = pd.read_csv('../data/credit_risk_dataset.csv')

except:
    data = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')
    data_to_predict = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
    data_ccrisk = pd.read_csv('/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv')


data_ccrisk.dropna(inplace=True)
data_ccrisk.drop_duplicates(inplace=True)

X_old = data.drop(['loan_status'], axis=1)
y_old = data['loan_status']

# Merge the dataframes
data_no_id = data.drop(['id'], axis=1)
merged_data = pd.concat([data_no_id, data_ccrisk], ignore_index=True)

X = merged_data.drop(['loan_status'], axis=1)
y = merged_data['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle=True, test_size=0.2, random_state=2024)

In [42]:
from imblearn.over_sampling import SMOTENC

categorical_columns = X.select_dtypes(include=['object']).columns

sm = SMOTENC(sampling_strategy=0.7, categorical_features=[X_train.columns.get_loc(col) for col in categorical_columns], random_state=2024)

X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [43]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

# Data processing
categorical_columns = X.select_dtypes(include=['object']).columns

categorical_ordinal = ['loan_grade']
categorical_onehot = categorical_columns.drop(categorical_ordinal)

log_columns = ['person_age', 'person_income', 'person_emp_length', 'loan_amnt']
log_columns = log_columns

numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
numerical_columns = numerical_columns.drop(log_columns)

def log_transform():
	return FunctionTransformer(np.log1p, validate=True)

preprocessor = ColumnTransformer(
	transformers=[
		('ordinal', OrdinalEncoder(), categorical_ordinal),
		('onehot', OneHotEncoder(), categorical_onehot),
		('scaler', StandardScaler(), numerical_columns),
		('log', log_transform(), log_columns)
	], remainder='passthrough'
	)

preprocessor.fit(X_train_res)

X_train_prep = preprocessor.transform(X_train_res)
X_test_prep = preprocessor.transform(X_test)

# Models

## XGBoost

In [44]:
def xgb_objective(trial):
    # Hyperparameter search space
    param = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 0.5),
        'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 3.0),
        'eval_metric': 'auc',  # AUC is the metric
        'objective': 'binary:logistic'  # Use binary logistic, probability outputs
    }

    # Initialize the model
    model = xgb.XGBClassifier(**param)

    # Fit the model
    model.fit(X_train_prep, y_train_res,
              eval_set=[(X_test_prep, y_test)],  
              verbose=False)

    # Predict proba for validation set
    y_pred_prob = model.predict_proba(X_test_prep)[:, 1]

    auc = roc_auc_score(y_test, y_pred_prob)
 
    return auc

# Study object to run the optimization. I want to maximize AUC
xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(xgb_objective, n_trials=50)

print(f"Best trial: {xgb_study.best_trial.params}")

[I 2024-10-12 17:00:42,630] A new study created in memory with name: no-name-2263b3b2-21cf-4d2b-b47c-4356feb80959
[I 2024-10-12 17:02:52,140] Trial 0 finished with value: 0.9507313241191518 and parameters: {'max_depth': 4, 'learning_rate': 0.24012624077728573, 'n_estimators': 121, 'subsample': 0.9481655671908854, 'colsample_bytree': 0.6383550771450843, 'gamma': 0.26860160585751036, 'lambda': 7.561187809487967, 'alpha': 7.164737627292461e-07, 'scale_pos_weight': 1.254144199791522}. Best is trial 0 with value: 0.9507313241191518.
[I 2024-10-12 17:18:20,806] Trial 1 finished with value: 0.9562505671326926 and parameters: {'max_depth': 9, 'learning_rate': 0.07011664926571008, 'n_estimators': 468, 'subsample': 0.8445727127932295, 'colsample_bytree': 0.5665594237930074, 'gamma': 0.23659380254993023, 'lambda': 0.07453919177780559, 'alpha': 1.5131321925272251e-06, 'scale_pos_weight': 2.695944515997298}. Best is trial 1 with value: 0.9562505671326926.
[I 2024-10-12 17:19:44,929] Trial 2 finishe

Best trial: {'max_depth': 6, 'learning_rate': 0.08218480696485447, 'n_estimators': 419, 'subsample': 0.9598787494275235, 'colsample_bytree': 0.7202079398862329, 'gamma': 0.20887266737138027, 'lambda': 5.603302128369934e-05, 'alpha': 1.7764720274807023e-05, 'scale_pos_weight': 1.0084836089316636}


In [45]:
xgb_best_params = xgb_study.best_trial.params
xgb_best_score = xgb_study.best_trial.value
xgb_best_score

0.9576538239093483

prev: 0.9576051262733373

## CatBoostClassifier

In [46]:
from catboost import CatBoostClassifier

def cat_objective(trial):

    # Define the hyperparameter search space

    param = {
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'iterations': trial.suggest_int('iterations', 200, 1000),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 3.0),
        'eval_metric': 'AUC',  
        'logging_level': 'Silent',  # Disable CatBoost output
        'task_type': 'CPU',  # Try using GPU
        'use_best_model': True
    }

    # Initialize the CatBoost model with the trial's parameters
    model = CatBoostClassifier(**param)

    # Fit the model
    model.fit(X_train_prep, y_train_res,
              eval_set=[(X_test_prep, y_test)],
              early_stopping_rounds=20,  # Early stopping to prevent overfitting
              verbose=False)

    # Get predictions and calculate AUC score
    y_pred_prob = model.predict_proba(X_test_prep)[:, 1]  # Probabilities for class 1

    auc = roc_auc_score(y_test, y_pred_prob)

    return auc
    
cat_study = optuna.create_study(direction='maximize')  # We want to maximize AUC
cat_study.optimize(cat_objective, n_trials=50)

# Output the best trial
print(f"Best trial: {cat_study.best_trial.params}")

[I 2024-10-12 17:56:15,040] A new study created in memory with name: no-name-c96b2107-4dff-4ecf-9ec1-bfef779afd5e
[I 2024-10-12 17:56:31,249] Trial 0 finished with value: 0.9460281483465679 and parameters: {'depth': 7, 'learning_rate': 0.04709665247569972, 'iterations': 469, 'l2_leaf_reg': 7.332825434593331e-06, 'border_count': 52, 'bagging_temperature': 0.5280668145234195, 'random_strength': 0.5146747469991674, 'scale_pos_weight': 2.51703942972807}. Best is trial 0 with value: 0.9460281483465679.
[I 2024-10-12 17:56:45,078] Trial 1 finished with value: 0.9502261853049596 and parameters: {'depth': 10, 'learning_rate': 0.15253812626222452, 'iterations': 834, 'l2_leaf_reg': 0.1392973180005424, 'border_count': 217, 'bagging_temperature': 0.8802359268625483, 'random_strength': 0.45298208313982746, 'scale_pos_weight': 2.8698008362912844}. Best is trial 1 with value: 0.9502261853049596.
[I 2024-10-12 17:56:55,805] Trial 2 finished with value: 0.9549404432529158 and parameters: {'depth': 8, '

Best trial: {'depth': 5, 'learning_rate': 0.1258808559129393, 'iterations': 778, 'l2_leaf_reg': 2.8587004971729453, 'border_count': 255, 'bagging_temperature': 0.11191509656237325, 'random_strength': 0.1702409451238981, 'scale_pos_weight': 1.0934074973940175}


In [47]:
cat_best_params = cat_study.best_trial.params
cat_best_score = cat_study.best_trial.value
cat_best_score

0.959752593749994

prev: 0.9601213839193278

## LightGBM

In [48]:
# Objective function for LightGBM

def lgb_objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 10.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
    }

    lgb_model = lgb.LGBMClassifier(**param)
  

    # Train the model
    lgb_model.fit(X_train_prep, y_train_res,
                  eval_set=[(X_test_prep, y_test)],
                  eval_metric='auc'
                  )

    # Predict and evaluate AUC
    y_pred_prob = lgb_model.predict_proba(X_test_prep)[:, 1]
    auc = roc_auc_score(y_test, y_pred_prob)

    return auc

# Optimize the objective function
lgb_study = optuna.create_study(direction='maximize')
lgb_study.optimize(lgb_objective, n_trials=50)

# Best parameters and AUC
print("Best LGBM Params: ", lgb_study.best_trial.params)
print("Best AUC for LGBM: ", lgb_study.best_value)

[I 2024-10-12 18:06:56,281] A new study created in memory with name: no-name-e8022a83-b95d-435a-877e-aa60d3ee442c


[LightGBM] [Info] Number of positive: 40660, number of negative: 58086
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009097 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1390
[LightGBM] [Info] Number of data points in the train set: 98746, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.411764 -> initscore=-0.356680
[LightGBM] [Info] Start training from score -0.356680
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2024-10-12 18:06:59,891] Trial 0 finished with value: 0.9589032681429892 and parameters: {'learning_rate': 0.23234812867231913, 'num_leaves': 229, 'max_depth': 4, 'min_child_samples': 78, 'min_child_weight': 2.0800834013858247, 'subsample': 0.9196533275862899, 'colsample_bytree': 0.5949543801059094, 'n_estimators': 401}. Best is trial 0 with value: 0.9589032681429892.


[LightGBM] [Info] Number of positive: 40660, number of negative: 58086
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005646 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1390
[LightGBM] [Info] Number of data points in the train set: 98746, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.411764 -> initscore=-0.356680
[LightGBM] [Info] Start training from score -0.356680
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

[I 2024-10-12 18:07:03,351] Trial 1 finished with value: 0.9550560956984677 and parameters: {'learning_rate': 0.19318835418733074, 'num_leaves': 203, 'max_depth': 15, 'min_child_samples': 71, 'min_child_weight': 2.0758087890701855, 'subsample': 0.659930960298156, 'colsample_bytree': 0.9459028602251096, 'n_estimators': 170}. Best is trial 0 with value: 0.9589032681429892.


...

[LightGBM] [Info] Number of positive: 40660, number of negative: 58086
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008449 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1390
[LightGBM] [Info] Number of data points in the train set: 98746, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.411764 -> initscore=-0.356680
[LightGBM] [Info] Start training from score -0.356680
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2024-10-12 18:13:01,088] Trial 48 finished with value: 0.95617440085687 and parameters: {'learning_rate': 0.019520666648972902, 'num_leaves': 166, 'max_depth': 7, 'min_child_samples': 5, 'min_child_weight': 8.705872450362914, 'subsample': 0.5296797744855617, 'colsample_bytree': 0.9486807177027433, 'n_estimators': 957}. Best is trial 4 with value: 0.960004452761233.


[LightGBM] [Info] Number of positive: 40660, number of negative: 58086
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009940 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1390
[LightGBM] [Info] Number of data points in the train set: 98746, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.411764 -> initscore=-0.356680
[LightGBM] [Info] Start training from score -0.356680
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2024-10-12 18:13:07,721] Trial 49 finished with value: 0.9594052812154096 and parameters: {'learning_rate': 0.06687904433386209, 'num_leaves': 281, 'max_depth': 5, 'min_child_samples': 9, 'min_child_weight': 3.446901037706915, 'subsample': 0.5611868602191304, 'colsample_bytree': 0.6728814664662276, 'n_estimators': 770}. Best is trial 4 with value: 0.960004452761233.


Best LGBM Params:  {'learning_rate': 0.029039945589354473, 'num_leaves': 100, 'max_depth': 15, 'min_child_samples': 10, 'min_child_weight': 3.352476848486978, 'subsample': 0.543215891847535, 'colsample_bytree': 0.5639949088196425, 'n_estimators': 930}
Best AUC for LGBM:  0.960004452761233


In [49]:
lgb_best_params = lgb_study.best_trial.params
lgb_best_score = lgb_study.best_value
lgb_best_score

0.960004452761233

prev: 0.9607718460460408

## ExtraTreesClassifier

In [50]:
from sklearn.ensemble import ExtraTreesClassifier

def extratrees_objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0)
    }

    model = ExtraTreesClassifier(**param)
    score = cross_val_score(model, X_train_prep, y_train_res, cv=3, scoring='roc_auc').mean()

    return score

extratrees_study = optuna.create_study(direction='maximize')
extratrees_study.optimize(extratrees_objective, n_trials=50, timeout=600)

print('Best parameters for ExtraTrees:', extratrees_study.best_params)

[I 2024-10-12 18:13:07,768] A new study created in memory with name: no-name-6fce1c72-effa-4ab0-bc51-6412cbfe2871
[I 2024-10-12 18:14:59,487] Trial 0 finished with value: 0.952951338258325 and parameters: {'n_estimators': 446, 'max_depth': 9, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 0.6745876059339685}. Best is trial 0 with value: 0.952951338258325.
[I 2024-10-12 18:15:46,443] Trial 1 finished with value: 0.9516451245517682 and parameters: {'n_estimators': 230, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 0.49752693933422076}. Best is trial 0 with value: 0.952951338258325.
[I 2024-10-12 18:16:22,418] Trial 2 finished with value: 0.9559531449377087 and parameters: {'n_estimators': 226, 'max_depth': 18, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': 0.23802354942403522}. Best is trial 2 with value: 0.9559531449377087.
[I 2024-10-12 18:17:06,143] Trial 3 finished with value: 0.9253577820296481 and parameters: {'n_est

Best parameters for ExtraTrees: {'n_estimators': 311, 'max_depth': 18, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_features': 0.45302441774199387}


In [51]:
extratrees_best_params = extratrees_study.best_params
extratrees_best_score = extratrees_study.best_value
extratrees_best_score

0.967021916728157

Prev: 0.9298212453316017

## LogisticRegression

In [52]:
from sklearn.linear_model import LogisticRegression

def logreg_objective(trial):
    
    param = {
        'penalty': 'elasticnet',
        'solver' : 'saga',
        'C': trial.suggest_float('C', 1e-4, 1e2, log=True),
        'l1_ratio': trial.suggest_float('l1_ratio', 0, 1)
    }

    model = LogisticRegression(**param, max_iter=1000)
    score = cross_val_score(model, X_train_prep, y_train_res, cv=3, scoring='roc_auc').mean()
    return score

logreg_study = optuna.create_study(direction='maximize')
logreg_study.optimize(logreg_objective, n_trials=50, timeout=600)
print('Best parameters for Logistic Regression:', logreg_study.best_params)

[I 2024-10-12 18:23:28,714] A new study created in memory with name: no-name-f3b4f0a8-baab-4388-913f-58739ece9ee9
[I 2024-10-12 18:23:57,568] Trial 0 finished with value: 0.910028604105741 and parameters: {'C': 7.087962306346659, 'l1_ratio': 0.913463574135948}. Best is trial 0 with value: 0.910028604105741.
[I 2024-10-12 18:25:08,943] Trial 1 finished with value: 0.9100785579241907 and parameters: {'C': 0.11052673822097674, 'l1_ratio': 0.890464449999595}. Best is trial 1 with value: 0.9100785579241907.
[I 2024-10-12 18:25:53,636] Trial 2 finished with value: 0.9099666096774112 and parameters: {'C': 0.03573478732081317, 'l1_ratio': 0.8524898855620824}. Best is trial 1 with value: 0.9100785579241907.
[I 2024-10-12 18:26:11,392] Trial 3 finished with value: 0.9100244696225003 and parameters: {'C': 90.9695948736603, 'l1_ratio': 0.5012938506283703}. Best is trial 1 with value: 0.9100785579241907.
[I 2024-10-12 18:26:29,320] Trial 4 finished with value: 0.9100260129200578 and parameters: {'C

Best parameters for Logistic Regression: {'C': 0.11052673822097674, 'l1_ratio': 0.890464449999595}


In [61]:
logreg_best_params = logreg_study.best_params
logreg_best_params['penalty'] = 'elasticnet'
logreg_best_params['solver'] = 'saga'
logreg_best_score = logreg_study.best_value
logreg_best_score

0.9100785579241907

prev: 0.8814172446287084

## HistGradientBoostingClassifier

In [54]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

def histgb_objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'max_iter': trial.suggest_int('max_iter', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 50),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 50),
        'l2_regularization': trial.suggest_float('l2_regularization', 1e-5, 1.0, log=True)
    }

    model = HistGradientBoostingClassifier(**param)
    score = cross_val_score(model, X_train_prep, y_train_res, cv=3, scoring='roc_auc').mean()

    return score

histgb_study = optuna.create_study(direction='maximize')
histgb_study.optimize(histgb_objective, n_trials=50, timeout=600)
print('Best parameters for HistGradientBoosting:', histgb_study.best_params)

c:\Users\Denis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(
[I 2024-10-12 18:34:52,309] A new study created in memory with name: no-name-8392c466-4a35-4db9-a3fc-4540bb8fb7fd
[I 2024-10-12 18:35:24,857] Trial 0 finished with value: 0.9747227654330096 and parameters: {'learning_rate': 0.021227276215547236, 'max_iter': 277, 'max_depth': 11, 'min_samples_leaf': 10, 'max_leaf_nodes': 50, 'l2_regularization': 0.17349418473479641}. Best is trial 0 with value: 0.9747227654330096.
[I 2024-10-12 18:35:54,083] Trial 1 finished with value: 0.9761508208268715 and parameters: {'learning_rate': 0.02340867799063854, 'max_iter': 442, 'max_depth': 6, 'min_samples_leaf': 29, 'max_leaf_nodes':

Best parameters for HistGradientBoosting: {'learning_rate': 0.0973302436339639, 'max_iter': 350, 'max_depth': 20, 'min_samples_leaf': 21, 'max_leaf_nodes': 40, 'l2_regularization': 2.024705049363233e-05}


In [55]:
histgb_best_params = histgb_study.best_trial.params
histgb_best_score = histgb_study.best_trial.value
histgb_best_score

0.9807639747945546

Prev: 0.9519987809872763

## KNN

In [56]:
from sklearn.neighbors import KNeighborsClassifier

def knn_objective(trial):
    param = {
        'n_neighbors': trial.suggest_int('n_neighbors', 3, 15),
        'leaf_size': trial.suggest_int('leaf_size', 20, 50),
        'p': trial.suggest_categorical('p', [1, 2]),  # Minkowski distance parameter
        'weights': trial.suggest_categorical('weights', ['uniform', 'distance'])
    }

    model = KNeighborsClassifier(**param)
    score = cross_val_score(model, X_train_prep, y_train_res, cv=3, scoring='roc_auc').mean()

    return score

knn_study = optuna.create_study(direction='maximize')
knn_study.optimize(knn_objective, n_trials=50, timeout=600)
print('Best parameters for KNN:', knn_study.best_params)
print('Best score for KNN:', knn_study.best_value)

[I 2024-10-12 18:44:53,727] A new study created in memory with name: no-name-07cd8edb-4def-4ed4-bb2d-c9e54ffb47aa
[I 2024-10-12 18:46:21,249] Trial 0 finished with value: 0.9512896923582271 and parameters: {'n_neighbors': 12, 'leaf_size': 46, 'p': 1, 'weights': 'uniform'}. Best is trial 0 with value: 0.9512896923582271.
[I 2024-10-12 18:46:48,600] Trial 1 finished with value: 0.9439928737368423 and parameters: {'n_neighbors': 6, 'leaf_size': 27, 'p': 2, 'weights': 'distance'}. Best is trial 0 with value: 0.9512896923582271.
[I 2024-10-12 18:47:15,978] Trial 2 finished with value: 0.9513008939884001 and parameters: {'n_neighbors': 15, 'leaf_size': 45, 'p': 2, 'weights': 'distance'}. Best is trial 2 with value: 0.9513008939884001.
[I 2024-10-12 18:48:43,062] Trial 3 finished with value: 0.9347276531193683 and parameters: {'n_neighbors': 3, 'leaf_size': 31, 'p': 1, 'weights': 'uniform'}. Best is trial 2 with value: 0.9513008939884001.
[I 2024-10-12 18:50:10,639] Trial 4 finished with valu

Best parameters for KNN: {'n_neighbors': 10, 'leaf_size': 34, 'p': 1, 'weights': 'distance'}
Best score for KNN: 0.9541333615154279


In [57]:
knn_best_params = knn_study.best_trial.params
knn_best_score = knn_study.best_trial.value
knn_best_score

0.9541333615154279

Prev: 0.9003336325395043

# Stacking

In [63]:
from sklearn.ensemble import StackingClassifier

# Init base models

xgb_model = xgb.XGBClassifier(**xgb_best_params)
cat_model = CatBoostClassifier(**cat_best_params)
lgb_model = lgb.LGBMClassifier(**lgb_best_params)
extratrees_model = ExtraTreesClassifier(**extratrees_best_params)
logreg_model = LogisticRegression(**logreg_best_params)
histgb_model = HistGradientBoostingClassifier(**histgb_best_params)
knn_model = KNeighborsClassifier(**knn_best_params)

# Base and meta models
base_estimators = [
	('xgb', xgb_model),
	('cat', cat_model),
	('lgb', lgb_model),
	('extratrees', extratrees_model),
	('logreg', logreg_model),
	('histgb', histgb_model),
	('knn', knn_model)
]

meta_model = LogisticRegression()

# Stack

stack_model = StackingClassifier(estimators=base_estimators, final_estimator=meta_model, cv=5, n_jobs=-1)

stack_model.fit(X_train_prep, y_train_res)

StackingClassifier(cv=5,
                   estimators=[('xgb',
                                XGBClassifier(alpha=1.7764720274807023e-05,
                                              base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.7202079398862329,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None,
                                              gamma=0.20887266737138027,
                                              grow_policy=N...
                                                   l1_ratio=0.890464449999595,
                                                   penalty='elasticnet',
                                                   solver='saga')),
                               ('histgb',
                                HistGradientBoostingClassifier(l2_regularization=2.024705049363233e-05,
                                                               learning_rate=0.0973302436339639,
                                                               max_depth=20,
                                                               max_iter=350,
                                                               max_leaf_nodes=40,
                                                               min_samples_leaf=21)),
                               ('knn',
                                KNeighborsClassifier(leaf_size=34,
                                                     n_neighbors=10, p=1,
                                                     weights='distance'))],
                   final_estimator=LogisticRegression(), n_jobs=-1)

In [64]:
print('Stacking Classifier ROC AUC:', roc_auc_score(y_test, stack_model.predict_proba(X_test_prep)[:, 1]))

Stacking Classifier ROC AUC: 0.9584046412908215


Prev: 0.9614965682196932

In [ ]:
# Choose different base models

base_estimators = [
	('xgb', xgb_model),
	('cat', cat_model),
	('lgb', lgb_model),
	('histgb', histgb_model),
	('knn', knn_model)

]

meta_model = lgb.LGBMClassifier()

# Stack
stack_model_2 = StackingClassifier(estimators=base_estimators, final_estimator=meta_model, cv=3, n_jobs=-1)

stack_model_2.fit(X_train_prep, y_train_res)

print('Stacking Classifier ROC AUC:', roc_auc_score(y_test, stack_model_2.predict_proba(X_test_prep)[:, 1]))

# Submission

In [ ]:
smote = SMOTENC(sampling_strategy=0.3, categorical_features=[X.columns.get_loc(col) for col in categorical_columns], random_state=2024)
X_res, y_res = smote.fit_resample(X, y)

X_train_all = preprocessor.transform(X_res)

stack_model.fit(X_train_all, y_res)

In [23]:
import joblib

try:
    joblib.dump(stack_model_2, '../models/stack_model_07_smote.pkl')
except:
    joblib.dump(stack_model_2, 'stack_model_07_smote.pkl')
    
X_to_pred = preprocessor.transform(data_to_predict)
y_pred_submit = stack_model_2.predict_proba(X_to_pred)[:, 1]

submission = pd.DataFrame({'id': data_to_predict['id'], 'loan_status': y_pred_submit})
try:
    submission.to_csv('../submissions/stack_model_v2.csv', index=False)
except:
    submission.to_csv('stack_model_v2.csv', index=False)